In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time
from datetime import datetime

import matplotlib.pyplot as plt

from sqlalchemy import Column, MetaData, create_engine, text

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load data

In [2]:
df_stock = (
    pd.read_parquet(
        './research_data/EOD_stock.pq',
        columns=['ticker','TradingDay','Open','Close','PreClose','TradeStatus','Volume','FloatAShare']
    ) 
    .query('TradingDay.between("20160101","20231231")') 
    .sort_values(['ticker','TradingDay'])
)

df_stock.shape
df_stock.head()

(7316957, 8)

,ticker,TradingDay,Open,Close,PreClose,TradeStatus,Volume,FloatAShare
489,000001.SZ,20160104,12.00,11.33,11.99,-1.0,563497.87,1.180405e+10
490,000001.SZ,20160105,11.27,11.40,11.33,-1.0,663269.95,1.180405e+10
491,000001.SZ,20160106,11.42,11.53,11.40,-1.0,515706.44,1.180405e+10
492,000001.SZ,20160107,11.41,10.94,11.53,-1.0,174761.10,1.180405e+10
493,000001.SZ,20160108,11.21,11.12,10.94,-1.0,747527.58,1.180405e+10


In [3]:
# 按照股票代码和交易日期排序
df_stock.sort_values(by=['ticker', 'TradingDay'], inplace=True)

# 1.计算收盘价相对于前一天收盘价的变化百分比
df_stock['return'] = (df_stock['Close'] / df_stock['PreClose']) - 1

# 2.计算未来收益率，即下一个开盘价与当前开盘价之间的收益率
df_stock['Next_Open'] = df_stock.groupby('ticker')['Open'].shift(-1)
df_stock['next_ret'] = (df_stock['Next_Open'] / df_stock['Open']) - 1
df_stock['turnover'] = (df_stock['Volume'] / df_stock['FloatAShare'])
df_stock['stv'] = np.where(abs(df_stock['return'] )>= 0.1, df_stock['return'] * 100, df_stock['turnover'])

# 删除不再需要的Next_Open列
df_stock.drop(columns=['Next_Open'], inplace=True)

# 打印结果的前几行
df_stock.sort_values(by=['stv'])
df_stock


,ticker,TradingDay,Open,Close,PreClose,TradeStatus,Volume,FloatAShare,return,next_ret,turnover,stv
8450698,833914.BJ,20160531,0.01,0.01,5.00,-1.0,100.0,0.0,-0.998000,0.000000,inf,-99.800000
8466085,834415.BJ,20160121,1.00,1.00,18.88,-1.0,5810.0,0.0,-0.947034,29.000000,inf,-94.703390
8421666,833284.BJ,20160108,1.00,1.00,15.11,-1.0,10945.0,0.0,-0.933819,0.000000,inf,-93.381866
4857095,430047.BJ,20160108,1.30,1.38,14.90,-1.0,5110.0,0.0,-0.907383,10.538462,inf,-90.738255
8472857,834682.BJ,20160304,1.00,1.00,10.20,-1.0,5000.0,0.0,-0.901961,0.000000,inf,-90.196078
...,...,...,...,...,...,...,...,...,...,...,...,...
8611958,873833.BJ,20230621,7.80,7.80,7.80,-1.0,0.0,0.0,0.000000,0.000000,NaN,NaN
8611959,873833.BJ,20230626,7.80,7.80,7.80,-1.0,0.0,0.0,0.000000,0.000000,NaN,NaN
8611960,873833.BJ,20230627,7.80,7.80,7.80,-1.0,0.0,0.0,0.000000,0.000000,NaN,NaN
8611961,873833.BJ,20230628,7.80,7.80,7.80,-1.0,0.0,0.0,0.000000,-0.016667,NaN,NaN


,ticker,TradingDay,Open,Close,PreClose,TradeStatus,Volume,FloatAShare,return,next_ret,turnover,stv
489,000001.SZ,20160104,12.00,11.33,11.99,-1.0,563497.87,1.180405e+10,-0.055046,-0.060833,0.000048,0.000048
490,000001.SZ,20160105,11.27,11.40,11.33,-1.0,663269.95,1.180405e+10,0.006178,0.013310,0.000056,0.000056
491,000001.SZ,20160106,11.42,11.53,11.40,-1.0,515706.44,1.180405e+10,0.011404,-0.000876,0.000044,0.000044
492,000001.SZ,20160107,11.41,10.94,11.53,-1.0,174761.10,1.180405e+10,-0.051171,-0.017528,0.000015,0.000015
493,000001.SZ,20160108,11.21,11.12,10.94,-1.0,747527.58,1.180405e+10,0.016453,-0.018733,0.000063,0.000063
...,...,...,...,...,...,...,...,...,...,...,...,...
8611959,873833.BJ,20230626,7.80,7.80,7.80,-1.0,0.00,0.000000e+00,0.000000,0.000000,NaN,NaN
8611960,873833.BJ,20230627,7.80,7.80,7.80,-1.0,0.00,0.000000e+00,0.000000,0.000000,NaN,NaN
8611961,873833.BJ,20230628,7.80,7.80,7.80,-1.0,0.00,0.000000e+00,0.000000,-0.016667,NaN,NaN
8611962,873833.BJ,20230629,7.67,7.67,7.80,-1.0,7.00,0.000000e+00,-0.016667,0.000000,inf,inf


In [4]:
# 3.计算基准，即中证1000（代号“000852.SH”）的收益率
#1)导入数据
df_index = (
    pd.read_parquet(
        './research_data/EOD_index.pq', 
        columns=['StockIndexCode','TradingDay','PreClose','Open','Close']
    ) 
    .query('TradingDay.between("20160101","20231231")') 
    .sort_values(['StockIndexCode','TradingDay'])
)
#2)计算基准
df_index = df_index[df_index['StockIndexCode'] == '000852.SH'].copy()
df_index['Benchmark_return'] = df_index['Close'] / df_index['PreClose'] - 1

# 显示基准收益率的前几行以检查结果
df_index.head()

,StockIndexCode,TradingDay,PreClose,Open,Close,Benchmark_return
8266,000852.SH,20160104,10614.3785,10598.2715,9694.8363,-0.086632
8267,000852.SH,20160105,9694.8363,9140.6723,9452.0406,-0.025044
8268,000852.SH,20160106,9452.0406,9511.7340,9709.5692,0.027246
8269,000852.SH,20160107,9709.5692,9511.1414,8856.4914,-0.087859
8270,000852.SH,20160108,8856.4914,9049.7667,8876.2526,0.002231


In [5]:
# 接下几步是把df_index里的benchmarkreturn匹配到df_stock中
# 首先确保TradingDay列在两个DataFrame中都是datetime类型
df_stock['TradingDay'] = pd.to_datetime(df_stock['TradingDay'])
df_index['TradingDay'] = pd.to_datetime(df_index['TradingDay'])

# 然后根据TradingDay列合并两个DataFrame
df_merged = df_stock.merge(df_index[['TradingDay', 'Benchmark_return']], on='TradingDay', how='left')

df_merged.head()  # 显示合并后的DataFrame的前几行以进行检查

,ticker,TradingDay,Open,Close,PreClose,TradeStatus,Volume,FloatAShare,return,next_ret,turnover,stv,Benchmark_return
0,000001.SZ,2016-01-04,12.00,11.33,11.99,-1.0,563497.87,1.180405e+10,-0.055046,-0.060833,0.000048,0.000048,-0.086632
1,000001.SZ,2016-01-05,11.27,11.40,11.33,-1.0,663269.95,1.180405e+10,0.006178,0.013310,0.000056,0.000056,-0.025044
2,000001.SZ,2016-01-06,11.42,11.53,11.40,-1.0,515706.44,1.180405e+10,0.011404,-0.000876,0.000044,0.000044,0.027246
3,000001.SZ,2016-01-07,11.41,10.94,11.53,-1.0,174761.10,1.180405e+10,-0.051171,-0.017528,0.000015,0.000015,-0.087859
4,000001.SZ,2016-01-08,11.21,11.12,10.94,-1.0,747527.58,1.180405e+10,0.016453,-0.018733,0.000063,0.000063,0.002231


In [6]:
#4.计算惊恐度
df_merged['sigema'] = abs(df_merged['return'] - df_merged['Benchmark_return'])/((abs(df_merged['return'])+abs(df_merged['Benchmark_return']))+0.1)
#5.计算加权决策分
df_merged['jqjcf'] = df_merged['sigema'] * df_merged['return'] * df_merged['stv']
# 显示前几行以检查结果
df_merged


,ticker,TradingDay,Open,Close,PreClose,TradeStatus,Volume,FloatAShare,return,next_ret,turnover,stv,Benchmark_return,sigema,jqjcf
0,000001.SZ,2016-01-04,12.00,11.33,11.99,-1.0,563497.87,1.180405e+10,-0.055046,-0.060833,0.000048,0.000048,-0.086632,0.130694,-3.434333e-07
1,000001.SZ,2016-01-05,11.27,11.40,11.33,-1.0,663269.95,1.180405e+10,0.006178,0.013310,0.000056,0.000056,-0.025044,0.237933,8.260045e-08
2,000001.SZ,2016-01-06,11.42,11.53,11.40,-1.0,515706.44,1.180405e+10,0.011404,-0.000876,0.000044,0.000044,0.027246,0.114262,5.692599e-08
3,000001.SZ,2016-01-07,11.41,10.94,11.53,-1.0,174761.10,1.180405e+10,-0.051171,-0.017528,0.000015,0.000015,-0.087859,0.153489,-1.162826e-07
4,000001.SZ,2016-01-08,11.21,11.12,10.94,-1.0,747527.58,1.180405e+10,0.016453,-0.018733,0.000063,0.000063,0.002231,0.119831,1.248593e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7316952,873833.BJ,2023-06-26,7.80,7.80,7.80,-1.0,0.00,0.000000e+00,0.000000,0.000000,NaN,NaN,-0.019460,0.162902,NaN
7316953,873833.BJ,2023-06-27,7.80,7.80,7.80,-1.0,0.00,0.000000e+00,0.000000,0.000000,NaN,NaN,0.013440,0.118480,NaN
7316954,873833.BJ,2023-06-28,7.80,7.80,7.80,-1.0,0.00,0.000000e+00,0.000000,-0.016667,NaN,NaN,-0.004882,0.046547,NaN
7316955,873833.BJ,2023-06-29,7.67,7.67,7.80,-1.0,7.00,0.000000e+00,-0.016667,0.000000,inf,inf,0.006877,0.190572,-inf


In [7]:
# 确保DataFrame是按照股票代码和交易日排序的
df_merged.sort_values(by=['ticker', 'TradingDay'], inplace=True)

# 6.计算惊恐收益因子factor_mean
df_merged['factor_mean'] = df_merged.groupby('ticker')['jqjcf'].transform(
    lambda x: x.rolling(window=20, min_periods=1).mean()
)

df_merged['factor_mean'] = df_merged.groupby('ticker')['factor_mean'].transform(
    lambda x: np.where(x.index < 20, np.nan, x)
)#剔除区间内前20个交易日的因子（NA）

# 7.计算惊恐波动因子factor_std
df_merged['factor_std'] = df_merged.groupby('ticker')['jqjcf'].transform(
    lambda x: x.rolling(window=20, min_periods=1).std()
)

df_merged['factor_std'] = df_merged.groupby('ticker')['factor_std'].transform(
    lambda x: np.where(x.index < 20, np.nan, x)
)#剔除区间内前20个交易日的因子（NA）

# 8.计算原始惊恐度因子factor_score
df_merged['factor_score'] = 0.5 * df_merged['factor_mean'] + 0.5 * df_merged['factor_std']

df_merged.head(30)

,ticker,TradingDay,Open,Close,PreClose,TradeStatus,Volume,FloatAShare,return,next_ret,turnover,stv,Benchmark_return,sigema,jqjcf,factor_mean,factor_std,factor_score
0,000001.SZ,2016-01-04,12.00,11.33,11.99,-1.0,563497.87,1.180405e+10,-0.055046,-0.060833,0.000048,0.000048,-0.086632,0.130694,-3.434333e-07,NaN,NaN,NaN
1,000001.SZ,2016-01-05,11.27,11.40,11.33,-1.0,663269.95,1.180405e+10,0.006178,0.013310,0.000056,0.000056,-0.025044,0.237933,8.260045e-08,NaN,NaN,NaN
2,000001.SZ,2016-01-06,11.42,11.53,11.40,-1.0,515706.44,1.180405e+10,0.011404,-0.000876,0.000044,0.000044,0.027246,0.114262,5.692599e-08,NaN,NaN,NaN
3,000001.SZ,2016-01-07,11.41,10.94,11.53,-1.0,174761.10,1.180405e+10,-0.051171,-0.017528,0.000015,0.000015,-0.087859,0.153489,-1.162826e-07,NaN,NaN,NaN
4,000001.SZ,2016-01-08,11.21,11.12,10.94,-1.0,747527.58,1.180405e+10,0.016453,-0.018733,0.000063,0.000063,0.002231,0.119831,1.248593e-07,NaN,NaN,NaN
5,000001.SZ,2016-01-11,11.00,10.76,11.12,-1.0,732013.99,1.180405e+10,-0.032374,-0.015455,0.000062,0.000062,-0.075610,0.207880,-4.173474e-07,NaN,NaN,NaN
6,000001.SZ,2016-01-12,10.83,10.81,10.76,-1.0,561642.30,1.180405e+10,0.004647,0.005540,0.000048,0.000048,-0.002092,0.063131,1.395814e-08,NaN,NaN,NaN
7,000001.SZ,2016-01-13,10.89,10.71,10.81,-1.0,391709.48,1.180405e+10,-0.009251,-0.027548,0.000033,0.000033,-0.042970,0.221516,-6.800055e-08,NaN,NaN,NaN
8,000001.SZ,2016-01-14,10.59,10.77,10.71,-1.0,666314.54,1.180405e+10,0.005602,0.006610,0.000056,0.000056,0.041043,0.241674,7.642589e-08,NaN,NaN,NaN
9,000001.SZ,2016-01-15,10.66,10.46,10.77,-1.0,448202.14,1.180405e+10,-0.028784,-0.030019,0.000038,0.000038,-0.035496,0.040861,-4.465792e-08,NaN,NaN,NaN


In [8]:
eod_stack = pd.concat([
    # Left dataframe: 
    ## take "Close" as an example, "pivot().stack(dropna=True)" method generate a complete [ticker, TradingDay] index without TradingDay gaps for each ticker
    df_merged.pivot(index='ticker', columns='TradingDay', values='Close').stack(dropna=False).to_frame('Close'),
    # Right dataframe:
    ## concatenate other fields
    df_merged.set_index(['ticker','TradingDay']).drop(columns=['Close'])
], axis=1)

eod_stack

Close   Open  PreClose  TradeStatus     Volume  \
ticker    TradingDay                                                   
000001.SZ 2016-01-04  11.33  12.00     11.99         -1.0  563497.87   
          2016-01-05  11.40  11.27     11.33         -1.0  663269.95   
          2016-01-06  11.53  11.42     11.40         -1.0  515706.44   
          2016-01-07  10.94  11.41     11.53         -1.0  174761.10   
          2016-01-08  11.12  11.21     10.94         -1.0  747527.58   
...                     ...    ...       ...          ...        ...   
873833.BJ 2023-06-26   7.80   7.80      7.80         -1.0       0.00   
          2023-06-27   7.80   7.80      7.80         -1.0       0.00   
          2023-06-28   7.80   7.80      7.80         -1.0       0.00   
          2023-06-29   7.67   7.67      7.80         -1.0       7.00   
          2023-06-30   7.67   7.67      7.67         -1.0       0.00   

                       FloatAShare    return  next_ret  turnover       stv  \
ticker    TradingDay                                                         
000001.SZ 2016-01-04  1.180405e+10 -0.055046 -0.060833  0.000048  0.000048   
          2016-01-05  1.180405e+10  0.006178  0.013310  0.000056  0.000056   
          2016-01-06  1.180405e+10  0.011404 -0.000876  0.000044  0.000044   
          2016-01-07  1.180405e+10 -0.051171 -0.017528  0.000015  0.000015   
          2016-01-08  1.180405e+10  0.016453 -0.018733  0.000063  0.000063   
...                            ...       ...       ...       ...       ...   
873833.BJ 2023-06-26  0.000000e+00  0.000000  0.000000       NaN       NaN   
          2023-06-27  0.000000e+00  0.000000  0.000000       NaN       NaN   
          2023-06-28  0.000000e+00  0.000000 -0.016667       NaN       NaN   
          2023-06-29  0.000000e+00 -0.016667  0.000000       inf       inf   
          2023-06-30  0.000000e+00  0.000000       NaN       NaN       NaN   

                      Benchmark_return    sigema         jqjcf  factor_mean  \
ticker    TradingDay                                                          
000001.SZ 2016-01-04         -0.086632  0.130694 -3.434333e-07          NaN   
          2016-01-05         -0.025044  0.237933  8.260045e-08          NaN   
          2016-01-06          0.027246  0.114262  5.692599e-08          NaN   
          2016-01-07         -0.087859  0.153489 -1.162826e-07          NaN   
          2016-01-08          0.002231  0.119831  1.248593e-07          NaN   
...                                ...       ...           ...          ...   
873833.BJ 2023-06-26         -0.019460  0.162902           NaN          NaN   
          2023-06-27          0.013440  0.118480           NaN          NaN   
          2023-06-28         -0.004882  0.046547           NaN          NaN   
          2023-06-29          0.006877  0.190572          -inf          NaN   
          2023-06-30          0.012576  0.111713           NaN          NaN   

                      factor_std  factor_score  
ticker    TradingDay                            
000001.SZ 2016-01-04         NaN           NaN  
          2016-01-05         NaN           NaN  
          2016-01-06         NaN           NaN  
          2016-01-07         NaN           NaN  
          2016-01-08         NaN           NaN  
...                          ...           ...  
873833.BJ 2023-06-26         NaN           NaN  
          2023-06-27         NaN           NaN  
          2023-06-28         NaN           NaN  
          2023-06-29         NaN           NaN  
          2023-06-30         NaN           NaN  

[9784233 rows x 16 columns]

# Calculate factors
pre-processed before backtesting:
 
 - Step 1: Remove the exchange suffix from "ticker"

 - Step 2: Covert "TradingDay" to datetime format

Check the other two issues:
 - factor dataset by [ticker, TradingDay] in ascending order

 - Check inf,-inf or other extreme values before uploading

In [9]:
factor_lst = ['factor_mean','factor_std','factor_score']
factor_stack = eod_stack[factor_lst].copy().reset_index().sort_values(['ticker','TradingDay'])

In [10]:
# exclude stocks on Beijing Stock Exchange
factor_stack = factor_stack.query('~ticker.str.endswith("BJ")')

In [11]:
# Step 1: Remove the exchange suffix from "ticker"
# Step 2: Covert "TradingDay" to datetime format
factor_stack = factor_stack.assign(
    ticker=lambda df: df['ticker'].str[:6],
    TradingDay=lambda df: pd.to_datetime(df['TradingDay'])
)

In [12]:
# check issue: check if your factors have inf,-inf values
for factor_mean in factor_lst:
    factor_stack[factor_mean] = factor_stack[factor_mean].replace({np.inf: np.nan, -np.inf: np.nan})

for factor_std in factor_lst:
    factor_stack[factor_std] = factor_stack[factor_std].replace({np.inf: np.nan, -np.inf: np.nan})
    
for factor_score in factor_lst:
    factor_stack[factor_score] = factor_stack[factor_score].replace({np.inf: np.nan, -np.inf: np.nan})
    

In [13]:
factor_stack

,ticker,TradingDay,factor_mean,factor_std,factor_score
0,000001,2016-01-04,NaN,NaN,NaN
1,000001,2016-01-05,NaN,NaN,NaN
2,000001,2016-01-06,NaN,NaN,NaN
3,000001,2016-01-07,NaN,NaN,NaN
4,000001,2016-01-08,NaN,NaN,NaN
...,...,...,...,...,...
9381787,689009,2023-06-26,0.131314,0.587267,0.359290
9381788,689009,2023-06-27,0.131314,0.587267,0.359290
9381789,689009,2023-06-28,0.131316,0.587266,0.359291
9381790,689009,2023-06-29,0.131316,0.587266,0.359291


In [14]:
factor_stack.describe()

,TradingDay,factor_mean,factor_std,factor_score
count,9352656,6.987542e+06,6.982407e+06,6.982407e+06
mean,2019-09-29 22:51:12.158157824,2.491404e-01,4.671529e-01,3.248133e-01
min,2016-01-04 00:00:00,-3.011361e-04,0.000000e+00,-5.710736e-05
25%,2017-11-15 00:00:00,-1.996379e-08,3.266797e-07,1.613400e-07
50%,2019-09-25 00:00:00,7.564483e-08,1.263668e-06,6.565980e-07
75%,2021-08-11 00:00:00,9.009308e-07,5.996470e-06,3.285858e-06
max,2023-06-30 00:00:00,3.754296e+04,2.654348e+04,2.265868e+04
std,NaN,2.523534e+01,3.220870e+01,2.350901e+01


# Save factors
Save factors in EOD format (index=TradingDay, columns=ticker), so that the backtest system can read
 - Factor dataset format:
   
   - Make sure to sort by [ticker, TradingDay] in ascending order before converting into EOD format

   - EOD format: index=TradingDay, columns=ticker, values=factor_values



 - Note:

    - Remove the exchange suffix from "ticker", e.g. "600000.SH" -> "600000"
    
    - Covert "TradingDay" to datetime format

In [15]:
for factor_1 in ['factor_mean']:
    factor_stack.pivot(index='TradingDay', columns='ticker', values=factor_1).to_parquet('./DailyFactors/factor_mean_re.pq')

for factor_2 in ['factor_std']:
    factor_stack.pivot(index='TradingDay', columns='ticker', values=factor_2).to_parquet('./DailyFactors/factor_std_re.pq')

for factor_3 in ['factor_score']:
    factor_stack.pivot(index='TradingDay', columns='ticker', values=factor_3).to_parquet('./DailyFactors/factor_score_re.pq')

# Backtest
 - Turn to "step2_simple_backtest.ipynb"